# Text to Speech

When solving some problem with deep learning in practice, you search the Web for the latest paper that solves that task, and take its implementation from GitHub. However, often there is no code, so being able to **reimplement a paper** is a vital skill. You will likely have to do it in your course project, and we are going to practice it in this assignment.

Let's focus on the task of text-to-speech (**TTS**) synthesis.

![](https://user-images.githubusercontent.com/9570420/81783573-392ed600-9504-11ea-98da-86ac05457c29.png)

## Data

 Intuitively, in the real world your dataset would be a set of pairs:

* text (string);
* **target**: raw audio of a person saying `text` (array of amplitude values sampled e.g. 44100 times per second — see an example plotted above).

We give you a simplified problem statement, with one dataset sample being an utterance described by

* list of [ARPAbet phonemes](http://www.speech.cs.cmu.edu/cgi-bin/cmudict#phones);
* *phoneme alignment*, i.e. start time (frame) and duration for each phoneme;
* **target**: [mel spectrogram](https://pytorch.org/audio/transforms.html#torchaudio.transforms.MelSpectrogram) of a person saying text.

![image](https://user-images.githubusercontent.com/9570420/81795777-2a9cea80-9515-11ea-99eb-05915f803af1.png)

In [1]:
import datetime
from IPython.display import clear_output

import random
from pathlib import Path

import lj_speech
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

import scipy.io.wavfile

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchnlp.encoders.text import pad_tensor

import torchtext
import torchtext.datasets
import torchtext.data

from tqdm import trange, tqdm

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [2]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/Deep Learning 2020 -- Home Assignment 4")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgotten to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

In [3]:
# ![ ! -d "LJ-Speech-aligned" ] && unzip -q "{AUX_DATA_ROOT / 'LJ-Speech-aligned.zip'}"
# !git clone --recursive https://github.com/shrubb/waveglow.git -b denoiser-fix
# !wget -c https://api.ngc.nvidia.com/v2/models/nvidia/waveglow_ljs_256channels/versions/2/files/waveglow_256channels_ljs_v2.pt

Use this Python module to handle our dataset. It's documented, so when in doubt, use `help()` or read the code with `??lj_speech`.

There are also a couple of useful constants, check them with `?lj_speech`.

Here is an example datapoint:

In [4]:
def collate_fn(data):
    batch_size = len(data)
    max_sequence_length = max([len(elem['phonemes_code']) for elem in data]) + 1
    max_spectrogram_length = max([len(elem['spectrogram'][0]) for elem in data])
    phonemes = torch.zeros((batch_size, max_sequence_length))
    durations = torch.zeros((batch_size, max_sequence_length))
    spectrograms = torch.zeros((batch_size, len(data[0]['spectrogram']), max_spectrogram_length))
    
    seq_lengths = torch.zeros(batch_size)
    spect_lengths = torch.zeros(batch_size)
    
    for i, elem in enumerate(data):
        phonemes[i] = torch.cat((torch.tensor([PHONEMES_DICT[code] for code in elem['phonemes_code']]).float(),
                                 torch.ones(max_sequence_length - len(elem['phonemes_code']))))
#         durations[i] = torch.cat((torch.tensor(elem['phonemes_duration']).float(),
#                                   torch.tensor(max_spectrogram_length - sum(elem['phonemes_duration'])),
#                                   torch.zeros(max_sequence_length - len(elem['phonemes_duration']) - 1)))
        durations[i] = pad_tensor(torch.tensor(elem['phonemes_duration'] + [max_spectrogram_length - sum(elem['phonemes_duration'])]), 
                                  length=max_sequence_length)
        spectrograms[i] = pad_tensor(torch.tensor(elem['spectrogram']).t(), length=max_spectrogram_length).t()
        seq_lengths[i] = len(elem['phonemes_code'])
        spect_lengths[i] = len(elem['spectrogram'][0])
    
    return phonemes.long(), durations.long(), spectrograms, seq_lengths.long(), spect_lengths.long()

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    dtype = torch.cuda.FloatTensor
    print('Using GPU\n')
else:
    dtype = torch.FloatTensor
    device = torch.device("cpu")
    print('Not using GPU\n')

DATASET_ROOT = Path('LJ-Speech-aligned/')
train_dataset, val_dataset = lj_speech.get_dataset(DATASET_ROOT)

Using GPU



In [6]:
NUM_WORKERS = 10
BATCH_SIZE = 32

N_TOKENS = len(lj_speech.POSSIBLE_PHONEME_CODES)
PHONEMES_DICT = dict(zip(lj_speech.POSSIBLE_PHONEME_CODES, range(1, N_TOKENS + 1)))
PAU_IDX = PHONEMES_DICT['pau']

print('Dataset element keys: ', train_dataset[0].keys())
print(f'Possible phonemes num: {len(lj_speech.POSSIBLE_PHONEME_CODES)}\n')

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, 
                          shuffle=True, collate_fn=collate_fn, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, 
                        shuffle=True, pin_memory=True)

example_datapoint = train_dataset[666]
print(f"Datasets yield: {', '.join(example_datapoint.keys())}")
plt.imshow(example_datapoint['spectrogram']);

Dataset element keys:  dict_keys(['text', 'phonemes_start', 'phonemes_duration', 'phonemes_code', 'spectrogram'])
Possible phonemes num: 54

Datasets yield: text, phonemes_start, phonemes_duration, phonemes_code, spectrogram


To "play" spectrograms, including those that you will generate, we will use another deep learning algorithm called [WaveGlow](https://arxiv.org/abs/1811.00002). It converts mel spectrograms to audio.

Fortunately, there *is* code for it on GitHub, so you won't have to reimplement it 🙂 Still, if you haven't done so, I encourage you to watch a short [video](https://www.youtube.com/watch?v=CqFIVCD1WWo) about a famous paper that it's based on.

In [7]:
vocoder = lj_speech.Vocoder()

In [8]:
print(example_datapoint['text'])

example_spectrogram = torch.from_numpy(example_datapoint['spectrogram'])
audio = vocoder(example_spectrogram)
lj_speech.play_audio(audio)

sometime after march twenty-seven, but according to marina oswald, prior to april ten, nineteen sixty-three,


Finally, we have phonemes as inputs, but we'd like to synthesize arbitrary text. For that, there is a function `lj_speech.text_to_phonemes(text)`.

## Method

Reimplement the baseline method defined by the diagram below. It's derived from a simplified version of the [DurIAN paper](https://arxiv.org/abs/1909.01700).

Just in case, here are some of the most notable differences from DurIAN:
* Prosodic boundary markers aren't used (we don't have them labeled), and thus there's no 'skip states' exclusion of prosodic boundaries' hidden states.
* Style codes aren't used too (same).
* Simpler network architectures.
* No pre-net in decoder.
* No attention used in decoder.
* Decoder's recurrent cell outputs single spectrogram frame at a time.
* Decoder's recurrent cell isn't conditioned on its own outputs (isn't "autogressive").

![pipeline](https://user-images.githubusercontent.com/9570420/81863803-6f0ba300-9574-11ea-9f02-481c2bba81f0.png)

This picture is the simplified Figure 1 from the paper. **Use the paper as a reference**. If something is unclear from the diagram — and, in fact, some things are intentionally omitted — search answers in the paper (but remain aware of the differences/simplifications in the above diagram) and other papers that the authors cite. There will even be details that aren't even mentioned anywhere in the paper, so you'll have to guess by your intuition. For instance, DurIAN paper doesn't explain batch size and sampling strategy, and even if it did, it would be irrelevant since we use different data. Finally, search the Web and read others' code; however, if you copy-paste code, **cite it**.

Tips:
* Only compute loss on the 'original' parts of the spectrograms (don't include padding)!
* When using recurrent nets, [clip gradients' norm](https://pytorch.org/docs/stable/nn.html#torch.nn.utils.clip_grad_norm_).
* Since you will use recurrent nets, and the sequences in a batch will be of different lengths, you may find [sequence packing utility](https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch) useful.
* Remember that you can send images and audio into TensorBoard, too.

**Train this baseline so that the words are well-recognizable, and try to get rid of the "metallic", "tin sounding" voice.**

**Aim for something like this:**

In [9]:
lj_speech.play_audio(scipy.io.wavfile.read(AUX_DATA_ROOT / 'ok.wav')[1])

**On the other hand, here is an example of insufficient quality:**

In [10]:
lj_speech.play_audio(scipy.io.wavfile.read(AUX_DATA_ROOT / 'bad.wav')[1])

In [12]:
def train(model, train_loader, val_loader, n_epochs=50):
    date = datetime.datetime.now().strftime("%b-%d-%Y-%H:%M:%S")
    writer_train = SummaryWriter(f'runs/{date}/train')
    model.to(device)
    optimizer = Adam(model.parameters())
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in trange(n_epochs):
        for i, (phonemes, durations, spectrograms, seq_lengths, spect_lengths) in enumerate(train_loader):
            phonemes, durations, spectrograms = phonemes.to(device), \
                                                durations.to(device).unsqueeze(2), \
                                                spectrograms.to(device)
#             mask = torch.tensor([torch.cat((torch.ones((80, spect_lengths[i])), 
#                                             torch.zeros((80, len(spectrograms[0]) - spect_lengths[i])))) 
#                                  for i in range(BATCH_SIZE)])
            mask = (spectrograms == 0)
            preds = model(phonemes, durations)
            
            loss = torch.abs((preds[0].masked_fill(mask, 0) - spectrograms)) + \
                   torch.abs((preds[0] + preds[1]).masked_fill(mask, 0) - spectrograms)
            loss = loss.mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            writer_train.add_scalar('Loss', loss.item(), epoch * len(train_loader) + i)
        clear_output(wait=True)

In [13]:
class TonyNet(nn.Module):
    def __init__(self, n_tokens):
        super().__init__()
        self.encoder = TonyNetEncoder(n_tokens)
        self.alignment = TonyNetAlignment()
        self.decoder = TonyNetDecoder()
        
        
    def forward(self, phonemes, durations):
        encoded = self.encoder(phonemes)
        if not self.training:
            durations = TonyNetDurations(phonemes)  # TODO: handle lengths, maybe do it not here
        aligned = self.alignment(encoded, durations)
        decoded_pair = self.decoder(aligned)
        
        return decoded_pair
    
    
class TonyNetEncoder(nn.Module):
    def __init__(self, n_tokens, embed_size=256, hid_dim=256, dims=256):
        super().__init__()
        
        self.tok_embedding = nn.Embedding(n_tokens, embed_size)
        
        self.conv_block1 = nn.ModuleList([nn.Sequential(nn.Conv1d(embed_size, hid_dim, kernel_size=5, padding=2),
                                                        nn.ReLU(),
                                                        nn.BatchNorm1d(hid_dim),
                                                        nn.Dropout()),
                                          nn.Sequential(nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                                        nn.ReLU(),
                                                        nn.BatchNorm1d(hid_dim),
                                                        nn.Dropout()),
                                         ])
        
        self.conv_block1_res = nn.ModuleList([nn.Conv1d(embed_size, hid_dim, kernel_size=5, padding=2),
                                              nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2)])
        
        self.conv_block2 = nn.ModuleList([nn.Sequential(nn.Conv1d(hid_dim, hid_dim * 2, kernel_size=5, padding=2),
                                                        nn.ReLU(),
                                                        nn.BatchNorm1d(hid_dim * 2),
                                                        nn.Dropout()),
                                          nn.Sequential(nn.Conv1d(hid_dim * 2, hid_dim * 2, kernel_size=5, padding=2),
                                                        nn.ReLU(),
                                                        nn.BatchNorm1d(hid_dim * 2),
                                                        nn.Dropout()),
                                          nn.Sequential(nn.Conv1d(hid_dim * 2, hid_dim * 2, kernel_size=5, padding=2),
                                                        nn.ReLU(),
                                                        nn.BatchNorm1d(hid_dim * 2),
                                                        nn.Dropout()),
                                         ])
        
        self.conv_block2_res = nn.ModuleList([nn.Conv1d(hid_dim, hid_dim * 2, kernel_size=5, padding=2),
                                              nn.Conv1d(hid_dim * 2, hid_dim * 2, kernel_size=5, padding=2),
                                              nn.Conv1d(hid_dim * 2, hid_dim * 2, kernel_size=5, padding=2)])
        
        self.gru = nn.GRU(input_size=hid_dim * 2, hidden_size=hid_dim, bidirectional=True, batch_first=True)
        
    
    def forward(self, phonemes):
        out = self.tok_embedding(phonemes)
        out = torch.transpose(out, 1, 2)
        for i, conv_block in enumerate(self.conv_block1):
            out = conv_block(out) + self.conv_block1_res[i](out)

        for i, conv_block in enumerate(self.conv_block2):
            out = conv_block(out) + self.conv_block2_res[i](out)
            
        out = torch.transpose(out, 1, 2)
            
        out = self.gru(out)[0]
        
        return out

        
class TonyNetAlignment(nn.Module):       
    def forward(self, representations, durations):
        batch_size, seq_len, hid_dim = representations.size()
        seq_len_new = durations[0].sum()
        durations = durations.flatten()
        representations = representations.reshape(-1, hid_dim)
        representations = representations.repeat_interleave(durations, dim=0)
        positions = torch.cat([torch.linspace(0, 1, d, device=representations.device) for d in durations]).reshape(-1, 1)
        representations = torch.cat([representations, positions], dim=1)
        representations = representations.reshape(batch_size, seq_len_new, hid_dim + 1)

        return representations
    
    
class TonyNetDecoder(nn.Module):
    def __init__(self, input_size=513, hid_dim=256):
        super().__init__()
        self.gru = nn.GRU(input_size=input_size, num_layers=2, hidden_size=hid_dim, batch_first=True)
        self.conv_out1 = nn.Conv1d(hid_dim, 80, kernel_size=1)
        self.post_net = nn.ModuleList([nn.Sequential(nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                                     nn.ReLU(),
                                                     nn.BatchNorm1d(hid_dim),
                                                     nn.Dropout(0)),
                                       nn.Sequential(nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                                     nn.ReLU(),
                                                     nn.BatchNorm1d(hid_dim * 2),
                                                     nn.Dropout(0)),
                                       nn.Sequential(nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                                     nn.ReLU(),
                                                     nn.BatchNorm1d(hid_dim),
                                                     nn.Dropout(0)),
                                       nn.Sequential(nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                                     nn.ReLU(),
                                                     nn.BatchNorm1d(hid_dim * 2),
                                                     nn.Dropout(0)),
                                       nn.Sequential(nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                                     nn.ReLU(),
                                                     nn.BatchNorm1d(hid_dim * 2),
                                                     nn.Dropout(0)),
                                       ])
        self.post_net_res = nn.ModuleList([nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                           nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                           nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                           nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2),
                                           nn.Conv1d(hid_dim, hid_dim, kernel_size=5, padding=2)])
        
        self.last = nn.Conv1d(hid_dim, 80, kernel_size=1)
        
        
    def forward(self, aligned_reprs):
        out = self.gru(aligned_reprs)[0]
        
        out = torch.transpose(out, 1, 2)
        net_out1 = self.conv_out1(out)
        
        for i, layer in enumerate(self.post_net_res):
            out = layer(out) + self.post_net_res[i](out)
            
        net_out2 = self.last(out)
        
        return net_out1, net_out2
        

class TonyNetDuration(nn.Module):
    def __init__(self, n_tokens, embed_size=256, hid_dim=256):
        super().__init__()
        
        self.tok_embedding = nn.Embedding(n_tokens, embed_size)
        
        self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hid_dim, num_layers=3, 
                            batch_first=True, bidirectional=True)
        
        self.dense = nn.Linear(hid_dim * 2, 1)
        
        
    def forward(self, phonemes):
        embedding = self.tok_embedding(phonemes)  # TODO probably need to transpose
        out = self.lstm(embedding)
        out = self.dense(out[0])
        
        return out  # TODO clamp by zero

In [14]:
model = TonyNet(N_TOKENS + 1)
train(model, train_loader, val_loader)

  4%|▍         | 2/50 [18:28<7:31:08, 563.92s/it]

KeyboardInterrupt: 

### Training loss
![Train loss model](https://drive.google.com/uc?id=1CfYy8tP1a6ueAFHnQjrQB2lft-4kTktF)

In [16]:
def train_dur(model, train_loader, val_loader, n_epochs=50):
    date = datetime.datetime.now().strftime("%b-%d-%Y-%H:%M:%S")
    writer_train = SummaryWriter(f'runs/{date}/train_dur')
    model.to(device)
    optimizer = Adam(model.parameters())
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in trange(n_epochs):
        for i, (phonemes, durations, spectrograms, seq_lengths, spect_lengths) in enumerate(train_loader):
            phonemes, durations, spectrograms = phonemes.to(device), \
                                                durations.to(device).unsqueeze(2), \
                                                spectrograms.to(device)
            preds = model(phonemes)
            loss = criterion(preds, durations.float())  # TODO add mask
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            writer_train.add_scalar('Loss', loss.item(), epoch * len(train_loader) + i)
        clear_output(wait=True)


dur_model = TonyNetDuration(N_TOKENS + 1)
train_dur(dur_model, train_loader, val_loader)


 34%|███▍      | 17/50 [10:31<20:53, 37.99s/it]

KeyboardInterrupt: 

### Training loss
![Train loss durations](https://drive.google.com/uc?id=11FucRb1JOx7KJ5loQpTauau8LvN9eSfD)

### Comments
I didn't manage to do task till the end. As we can see, model learn **something** but of course we can't judge perfomance anyhow just on the decreasing of the loss value. I know that students who achieved some more or less ok results had loss values for the spectrogram generating model around 1 or even less, but the best one that I achieved is about 1.5.

I the slack it was stated to outline the rest of the needed steps to improve the solution if we do not finish it. I've left some `TODO` comments in the code above to indicate that.

## Improving the Method

When you have the baseline working, seek the ways to improve quality and to decrease loss. Look into the paper and the relevant/neighbouring literature. Also, follow your intuition, e.g. for changing model complexity or regularization. For example, you can (but aren't obliged to) try borrowing more tricks from DurIAN; a couple of examples for inspiration:

* Make decoder recurrent cell autoregressive as in the paper: use its output as its input at the next preiction step.
* Add pre-net to decoder.
* Add attention to decoder.
* Use more complex sub-network architectures (e.g. use CBHG).
* Employ tricks from Tacotron {[1](https://arxiv.org/abs/1703.10135),[2](https://arxiv.org/abs/1712.05884)} — papers that DurIAN is based upon, e.g. [zoneout](https://arxiv.org/abs/1606.01305) in RNNs.

Write a report on your journey to better loss. Explain if you managed to perceptibly improve the quality and lower the loss, what you tried and why, what worked and what didn't. Include TensorBoard loss plots.

## Deliverables

1. This file, fully functional, reproducing all the reported results on "Run all". Out-of-the box `TextToSpeechSynthesizer` class (see interface below) and demo code of running it on arbitrary text.
2. Report on improving the baseline at the end of this file.
3. Two "checkpoint files" with weights for both of your models (baseline and the improved model).

## Grading

* **[7 points]** Baseline.
* **[3 points]** Improving of the baseline.

In [ ]:
class TextToSpeechSynthesizer:
    """
    When done, please fill this class. It should work out-of-the-box, i.e.
    have a simple interface, automatically load model weights,
    process text to speech at one command without errors etc.
    """
    def __init__(self, checkpoint_path, <any-other-arguments>):
        """
        Initialize anything you may need. For example, load model weights from disk.
        """
        pass
    
    def synthesize_from_text(self, text):
        """
        Synthesize text into voice.

        text:
            str
        
        return:
        audio:
            torch.Tensor or numpy.ndarray, shape == (1, t)
        """
        phonemes = lj_speech.text_to_phonemes(text)
        return self.synthesize_from_phonemes(phonemes)

    def synthesize_from_phonemes(self, phonemes, durations=None):
        """
        Synthesize phonemes into voice.

        phonemes:
            list of str
            ARPAbet phoneme codes.
        durations:
            list of int, optional
            Duration in spectrogram frames for each phoneme.
            If given, will be used for hard alignment in the model (like during
            training); otherwise, durations will be predicted by the duration
            model.
        
        return:
        audio:
            torch.Tensor or numpy.ndarray, shape == (1, t)
        """
        # Your code here

        spectrogram = ...
        return vocoder(spectrogram)

In [ ]:
# Example:
synthesizer = TextToSpeechSynthesizer("./baseline.pth", <any-other-arguments>)

In [ ]:
phonemes = "DH IH1 S pau pau pau IH1 Z pau pau pau S P AH1 R T AH1 AH1 AH1 pau".split()
lj_speech.play_audio(
    synthesizer.synthesize_from_phonemes(phonemes))

In [ ]:
text = "Pack my box with five dozen liquor jugs."
# text = "The five boxing wizards jump quickly."
# text = "The quick brown fox jumps over the lazy dog."
# text = "How about some school tech."
# text = "Last homework. We are in a deep trouble. No sleep tonight."

lj_speech.play_audio(
    synthesizer.synthesize_from_text(text))